<a href="http://www.calstatela.edu/centers/hipic"><img align="left" src="https://avatars2.githubusercontent.com/u/4156894?v=3&s=100"><image/>
</a>
<img align="right" alt="California State University, Los Angeles" src="http://www.calstatela.edu/sites/default/files/groups/California%20State%20University%2C%20Los%20Angeles/master_logo_full_color_horizontal_centered.svg" style="width: 360px;"/>

#    CIS5560 Term Project Tutorial 
##   PySpark Collaborative Filtering in Databricks

------
#### Authors: [Monika Mishra](https://www.linkedin.com/in/monika-mishra-8b2a4115/), [Amogh Mahesh](https://www.linkedin.com/in/amoghmahesh/), [Aakanksha Tasgaonkar](https://www.linkedin.com/in/aakanksha-tasgaonkar-272ba393/)

#### Instructor: [Jongwook Woo](https://www.linkedin.com/in/jongwook-woo-7081a85)

#### Date: 04/26/2019

## Collaborative Filtering
Collaborative filtering is a machine learning technique that predicts ratings awarded to items by users.

### Import the ALS class
In this exercise, you will use the Alternating Least Squares collaborative filtering algorithm to creater a recommender.

### Two types of user preferences:

__Explicit preference__ (also referred as "Explicit feedback"), such as "rating" given to item by users. Default for ALS

__Implicit preference__ (also referred as "Implicit feedback"), such as "view" and "buy" history.

### You will be using just the Expilicit preference in this tutorial

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit


### Load Source Data
The source data for the recommender is in one file.

Read csv file from DBFS (Databricks File Systems)

### Follow the direction to read your table after upload it to Data at the left frame

1. After _amazon_rec.csv_ file is added to the data of the left frame, create a table using the UI, especially, "Upload File"
1. Click "Preview Table to view the table" and Select the option as _amazon_rec.csv_ has a header as the first row: "First line is header"
1. __Change the data type__ of the table columns: (customer_id: int, rating: double, product_category: string)
1. When you click on create table button, remember the table name, for example, _amazon_

### Assign the table name to data, which is created at TODO 1, using Spark SQL 
#### _spark.sql("SELECT * FROM amazon")_, 
ratings_all = spark.sql("SELECT customer_id, product_category, star_rating FROM amazon")

In [8]:
ratings_all = spark.sql("SELECT customer_id, product_category, star_rating FROM amazon")
ratings_all.show(5)

+-----------+--------------------+-----------+
customer_id| product_category|star_rating|
+-----------+--------------------+-----------+
 16334773|Digital_Video_Dow...| 4.0|
 12167164| Music| 5.0|
 9946096| Toys| 4.0|
 52224424|Digital_Ebook_Pur...| 2.0|
 51712853| Video DVD| 3.0|
+-----------+--------------------+-----------+
only showing top 5 rows

### Count the number of rows

In [10]:
ratings_all.count()

Out[ 25 ]: 134372

### Drop if any duplicate rows present

In [12]:
ratings = ratings_all.drop_duplicates()
ratings.count()

Out[ 26 ]: 134372

### The ALS model requires all inputs to be numerical.
### Using StringIndexer to convert product_category to product_categoryIdx

In [14]:
new_rating = ratings
strIdx = StringIndexer(inputCol = "product_category", outputCol = "product_categoryIdx")
sm = strIdx.fit(new_rating)
new_rating = sm.transform(new_rating)
new_rating.show(10)

+-----------+--------------------+-----------+-------------------+
customer_id| product_category|star_rating|product_categoryIdx|
+-----------+--------------------+-----------+-------------------+
 46418534| Mobile_Apps| 5.0| 0.0|
 39659880| Mobile_Apps| 5.0| 0.0|
 23823498| Video DVD| 4.0| 3.0|
 51798019| Music| 4.0| 5.0|
 17736563| Music| 5.0| 5.0|
 42583279| Toys| 5.0| 7.0|
 15323466| Video DVD| 1.0| 3.0|
 45369689| Mobile_Apps| 5.0| 0.0|
 26246512|Digital_Music_Pur...| 5.0| 6.0|
 48175696|Digital_Ebook_Pur...| 4.0| 1.0|
+-----------+--------------------+-----------+-------------------+
only showing top 10 rows

### Prepare the Data
To prepare the data, split it into a training set and a test set.

In [16]:
data = new_rating.select("customer_id", "product_categoryIdx", "star_rating")
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("star_rating", "label")
test = splits[1].withColumnRenamed("star_rating", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 93997 Testing Rows: 40375

### Build the Recommender
In ALS, users and products are described by a small set of latent features (factors) that can be used to predict missing entries.

The ALS class is an estimator, so you can use its **fit** method to traing a model, or you can include it in a pipeline. Rather than specifying a feature vector and as label, the ALS algorithm requries a numeric user ID, item ID, and rating.

In [18]:
als = ALS(userCol="customer_id", itemCol="product_categoryIdx", ratingCol="label")

#### Add paramGrid and Validation

In [20]:
paramGrid = ParamGridBuilder() \
                    .addGrid(als.rank, [1, 5]) \
                    .addGrid(als.maxIter, [5, 10]) \
                    .addGrid(als.regParam, [0.3, 0.1, 0.01]) \
                    .addGrid(als.alpha, [2.0,3.0]) \
                    .build()



In [21]:
cv = TrainValidationSplit(estimator=als, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, trainRatio=0.8)
model = cv.fit(train)

### Test the Recommender
Now that you've trained the recommender, you can see how accurately it predicts known ratings in the test set.

In [23]:
prediction = model.transform(test)

# Remove NaN values from prediction (due to SPARK-14489) [1]
prediction = prediction.filter(prediction.prediction != float('nan'))

# Round floats to whole numbers
prediction = prediction.withColumn("prediction", F.abs(F.round(prediction["prediction"],0)))


In [24]:
prediction.join(new_rating, ["customer_id", "product_categoryIdx"]).select("customer_id", "product_category", "prediction", "trueLabel").show(50, truncate=False)

+-----------+----------------------+----------+---------+
customer_id|product_category |prediction|trueLabel|
+-----------+----------------------+----------+---------+
51351143 |Camera |3.0 |5.0 |
29733886 |Camera |4.0 |5.0 |
28319815 |Digital_Ebook_Purchase|5.0 |4.0 |
21563560 |Digital_Ebook_Purchase|5.0 |5.0 |
20193301 |Digital_Ebook_Purchase|6.0 |5.0 |
2045953 |Digital_Ebook_Purchase|6.0 |5.0 |
36743946 |Digital_Ebook_Purchase|5.0 |5.0 |
44736416 |Digital_Ebook_Purchase|8.0 |5.0 |
42637964 |Digital_Ebook_Purchase|3.0 |3.0 |
11276418 |Digital_Ebook_Purchase|5.0 |5.0 |
12166877 |Digital_Ebook_Purchase|5.0 |5.0 |
52796842 |Digital_Ebook_Purchase|4.0 |2.0 |
10532597 |Digital_Ebook_Purchase|8.0 |5.0 |
31781524 |Digital_Ebook_Purchase|5.0 |1.0 |
28895654 |Digital_Ebook_Purchase|5.0 |3.0 |
30427132 |Digital_Ebook_Purchase|3.0 |5.0 |
11763926 |Digital_Ebook_Purchase|5.0 |5.0 |
35025567 |Digital_Ebook_Purchase|8.0 |5.0 |
41785508 |Digital_Ebook_Purchase|5.0 |5.0 |
34240303 |Digital_Ebook_Purchase|5.0 |5.0 |
51390083 |Digital_Ebook_Purchase|4.0 |5.0 |
16676134 |Digital_Ebook_Purchase|4.0 |5.0 |
52191364 |Digital_Ebook_Purchase|6.0 |2.0 |
52895956 |Digital_Ebook_Purchase|4.0 |5.0 |
26699218 |Digital_Ebook_Purchase|8.0 |5.0 |
36549733 |Digital_Ebook_Purchase|8.0 |5.0 |
49805190 |Digital_Ebook_Purchase|5.0 |5.0 |
53002945 |Digital_Ebook_Purchase|3.0 |5.0 |
52120366 |Digital_Ebook_Purchase|4.0 |5.0 |
11099023 |Digital_Ebook_Purchase|8.0 |5.0 |
25130238 |Digital_Ebook_Purchase|8.0 |5.0 |
11273962 |Digital_Ebook_Purchase|6.0 |4.0 |
17741109 |Digital_Ebook_Purchase|4.0 |1.0 |
49245549 |Digital_Ebook_Purchase|6.0 |5.0 |
23258960 |Digital_Ebook_Purchase|6.0 |5.0 |
37860085 |Digital_Ebook_Purchase|5.0 |5.0 |
15329856 |Digital_Ebook_Purchase|6.0 |5.0 |
36401051 |Digital_Ebook_Purchase|3.0 |3.0 |
12459952 |Digital_Ebook_Purchase|5.0 |5.0 |
25437661 |Digital_Ebook_Purchase|5.0 |4.0 |
52786848 |Digital_Ebook_Purchase|5.0 |4.0 |
34559407 |Digital_Ebook_Purchase|6.0 |4.0 |
50762340 |Digital_Ebook_Purchase|4.0 |5.0 |
47595292 |Digital_Ebook_Purchase|5.0 |5.0 |
13012404 |Digital_Ebook_Purchase|3.0 |4.0 |
10641330 |Digital_Ebook_Purchase|5.0 |3.0 |
52501475 |Digital_Ebook_Purchase|5.0 |5.0 |
45071815 |Digital_Ebook_Purchase|8.0 |5.0 |
42781533 |Digital_Ebook_Purchase|4.0 |4.0 |
42879823 |Digital_Ebook_Purchase|2.0 |5.0 |
+-----------+----------------------+----------+---------+
only showing top 50 rows

#### RegressionEvaluator
Calculate RMSE using RegressionEvaluator.

__NOTE:__ make sure to set [predictionCol="prediction"]

In [26]:
# RegressionEvaluator: predictionCol="prediction", metricName="rmse"

evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)

print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.7290606442555978

**Reference**

1. Predicting Song Listens Using Apache Spark, https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3175648861028866/48824497172554/657465297935335/latest.html
1. Dataset Link : https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_US_v1_00.tsv.gz